In [1]:
## Libraries
%matplotlib inline
import math
import numpy as np
import matplotlib.pyplot as plt
import pv_model_class
import cvxpy as cp
import pandas as pd

In [2]:
# Constants and variables

Iscn = 8.21 #8.7 # nominal short circuit current
Vocn = 32.9 #37.7 # nominal open circuit voltage
Imp = 7.61 #8.2 # array current at MPP
Vmp = 26.3 #30.1 # array voltage at MPP
Pmax_e = Vmp * Imp # experimental array max power output
Kv = -0.123 #-0.32/100 * Vocn # voltage temperature coefficient
Ki = 0.0032 #-0.032/100 * Iscn # current temperature coefficient
Ns = 54 #60. # number of series cells
Gn = 1000. # nominal irradiance
G = Gn
Tn = 25. + 273.15 # nominal operating temperature
T = Tn

Egap = 1.8 * 10 ** -19 # Bandgap of silicon (silicio cristalino)

err = 0.0001
inverter = 0.95
array_dim = [1, 1]

In [3]:
### Linear Program Example

m = 15
n = 10
np.random.seed(1)
s0 = np.random.randn(m)
lamb0 = np.maximum(-s0, 0)
s0 = np.maximum(s0, 0)
x0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A @ x0 + s0
c = -A.T @ lamb0

# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize(c.T@x),
                 [A @ x <= b])
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution is")
print(prob.constraints[0].dual_value)


## use '@' when doing multiplication between vectors and matrices -- turns them into scalars
## use '*' when doing multiplication between scalars and vectors or matrices


The optimal value is -15.220912605552861
A solution x is
[-1.10133381 -0.16360111 -0.89734939  0.03216603  0.6069123  -1.12687348
  1.12967856  0.88176638  0.49075229  0.8984822 ]
A dual solution is
[6.98805172e-10 6.11756416e-01 5.28171747e-01 1.07296862e+00
 3.93759300e-09 2.30153870e+00 4.25704434e-10 7.61206896e-01
 8.36906030e-09 2.49370377e-01 1.30187120e-09 2.06014070e+00
 3.22417207e-01 3.84054343e-01 1.59493839e-09]


## Helpful links:
1. https://www.cvxpy.org/tutorial/functions/index.html
2. https://www.cvxpy.org/examples/basic/linear_program.html


# charging constraints
p_cha >= 0
p_cha <= p_lim

# dischargin constraints
p_dis >= 0
p_dis <= p_lim

# SOC constraints
J >= 0
J <= J_max


In [26]:
## getting LMP data, etc

df = pd.read_csv("201801_dalmp_TH_NP15_GEN-APND.csv")
lmp = df["LMP"]
lmp_oneday = lmp[17:41]
lmp_oneday

# range from 8 to 23 hours (index 0 to 15, length of 16)
df['INTERVALSTARTTIME_GMT'][17]

'2018-01-02T01:00:00-00:00'

In [27]:
p_dem_df = pd.read_csv("USA_WA_Seattle-Tacoma.727930_TMY2.csv")
p_dem_df.head()
p_dem_df['Date/Time'][47]

' 01/02  24:00:00'

In [29]:
### constants

# time step currently is ONE HOUR

# power limit
p_lim = 5 * 10**3
# state of charge max
J_max = 20 * 10 **3
# charge efficiency
cha_eff = 0.85
# discharge efficiency
dis_eff = 0.15
# power demand
p_dem = p_dem_df["Electricity:Facility [kWh](Hourly)"][23:47]
# power supplied by pv
# calculated for Jan 1, using TMYFormatting.ipynb
p_pv = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
        0.04458429583591524, 16.005879953384856, 
        17.073777706000868, 40.99387471884097, 
        22.73501183321906, 16.011503472109514, 
        12.890191919606584, 2.469797077496577, 
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

## Length is 16 since the LMP data starts at 8am and goes to 11pm
p_cha = cp.Variable(24)
p_dis = cp.Variable(24)

# convert pandas Series into numpy array
_lmp = lmp_oneday.to_numpy()
p_dem = p_dem.to_numpy()

p_net = p_dem - p_pv + p_cha - p_dis

# state of charge (SOC)
J = cha_eff * p_cha - p_dis / dis_eff

prob = cp.Problem(cp.Minimize(_lmp.T @ p_net),
                 [p_cha >= 0, p_cha <= p_lim, p_dis >= 0, p_dis <= p_lim, J >= 0, J <= J_max])
prob.solve()
print("\nThe optimal value is", prob.value)
print("A solution p_cha is")
print(p_cha.value)
print("A solution p_dis is")
print(p_dis.value)



The optimal value is -3705.20153163716
A solution p_cha is
[-5.58696959e-12 -5.38360188e-12 -5.62126701e-12 -4.74593548e-12
 -4.40908749e-12 -4.15312665e-12 -3.85389097e-12 -3.89737904e-12
 -3.78198154e-12 -3.69202125e-12 -3.58777978e-12 -3.77621905e-12
 -3.96303278e-12 -4.73110499e-12 -5.38701998e-12 -4.19643399e-12
 -3.87430777e-12 -3.70160265e-12 -3.61458772e-12 -3.61540349e-12
 -3.78592137e-12 -3.84252801e-12 -4.22901991e-12 -4.73687140e-12]
A solution p_dis is
[1.49037314e-13 1.43917875e-13 1.49899546e-13 1.27803147e-13
 1.19311268e-13 1.12897413e-13 1.05456408e-13 1.06533457e-13
 1.03679021e-13 1.01462060e-13 9.89029977e-14 1.03536787e-13
 1.08162412e-13 1.27428399e-13 1.44004025e-13 1.13979718e-13
 1.05961864e-13 1.01697819e-13 9.95600593e-14 9.95800654e-14
 1.03776283e-13 1.05175250e-13 1.14794898e-13 1.27574102e-13]
